<h1 align="center"> COMP 7630 Group Project </h1>

## 1. Preparation

Install libs

In [6]:
%pip install wordcloud
%pip install -U spacy
%pip install pandas
%pip install bs4
%pip install requests
%pip install joblib
%pip install matplotlib
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


     --------------------------------------- 12.8/12.8 MB 59.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Import libs

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re
import requests
import matplotlib.pyplot as plt

## 2. Data Acquirement 

In [ ]:
def get_movies(name):
    url = f'https://www.rottentomatoes.com/browse/{name}/?page=5'
    # url = 'https://www.rottentomatoes.com/browse/movies_in_theaters/?page=5'
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'content-type': 'text/html'
    }
    res = requests.get(url=url, headers=headers)
    pattern = r'alt="(.*?)"\s*class="posterImage"'
    matches = re.findall(pattern, res.text)
    print(len(matches), 'Movies')
    if len(matches) > 0:
        for i in range(len(matches)):
            matches[i] = matches[i].lower()
            matches[i] = re.sub(r"[^a-zA-Z0-9]+", "_", matches[i])
    return matches

In [ ]:
def get_comments_movies(name):
    url = f'https://www.rottentomatoes.com/m/{name}/reviews'
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'content-type': 'text/html',
    }

    res = requests.get(url=url, headers=headers)

    # Define a regular expression pattern to match the phrase "review-quote"
    pattern = r'review-quote">(.*?)<\/p>'
    comments = re.findall(pattern, res.text)
    if len(comments) > 0:
        for i in range(len(comments)):
            comments[i] = re.sub(r'\d+', '', comments[i])
            comments[i] = re.sub(r'[&#;]', '', comments[i])
        # print(comments)
    pattern = r'hide" state="([^"]+)"'
    labels = re.findall(pattern, res.text)
    # print(labels)
    return comments, labels

In [ ]:
def movie_data():
    infor_dict = {
        'movie_name': [],
        'movie_comment': [],
        'movie_label': []
    }
    movies = []
    movies_type = [
        'movies_at_home',
        'movies_at_home/sort:popular',
        'movies_at_home/sort:audience_highest',
        'movies_at_home/sort:audience_lowest',
        'movies_at_home/sort:critic_highest',
        'movies_at_home/sort:critic_lowest',
        'movies_in_theaters',
        'movies_in_theaters/sort:popular',
        'movies_in_theaters/sort:audience_highest',
        'movies_in_theaters/sort:audience_lowest',
        'movies_in_theaters/sort:critic_highest',
        'movies_in_theaters/sort:critic_lowest',
        'tv_series_browse',
        'tv_series_browse/sort:popular',
        'tv_series_browse/sort:audience_highest',
        'tv_series_browse/sort:audience_lowest',
        'tv_series_browse/sort:critic_highest',
        'tv_series_browse/sort:critic_lowest'
    ]
    for name in movies_type:
        movies += get_movies(name)
    print(len(movies), 'total')
    for i in range(len(movies)):
        comments, labels = get_comments_movies(movies[i])
        if i % 100 == 0:
            print(i, 'movies be processed')
        for j in range(len(comments)):
            infor_dict['movie_name'].append(movies[i])
            infor_dict['movie_comment'].append(comments[j])
            infor_dict['movie_label'].append(labels[j])

    df = pd.DataFrame(infor_dict)
    df.to_csv('movies.csv', index=False)


In [ ]:
def get_games(name):
    url = f'https://store.steampowered.com/saleaction/ajaxgetsaledynamicappquery?cc=HK&l=english&clanAccountID=41316928&clanAnnouncementGID=3128313422564004283&flavor=popularpurchased&start={name}&count=150&tabuniqueid=8&sectionuniqueid=93094&return_capsules=true&origin=https:%2F%2Fstore.steampowered.com&bForceUseSaleTag=true&strContentHubType=freetoplay&strTabFilter=&strSectionFilter=%7B%22type%22:0,%22bNegated%22:true,%22rgSubexpressions%22:[%7B%22type%22:7,%22value%22:%22dlc%22%7D]%7D&bPrioritizeDiscounts=false'
    headers = {
        'Content-Type': 'application/json'
    }
    res = requests.get(url=url, headers=headers)
    res_dict = res.json()
    games = res_dict['appids']
    return games

In [ ]:
def get_comments_games(name):
    url = f'https://steamcommunity.com/app/{name}/reviews/?browsefilter=toprated&snr=1_5_100010_&filterLanguage=english'
    headers = {
        'Content-Type': 'text/html',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    }
    res = requests.get(url=url, headers=headers)
    # print(res.text)
    pattern = r'</div>\s+(.*?)\s+</div>\s+</div>\s+<div class="UserReviewCardContent_Footer">'
    comments = re.findall(pattern, res.text)
    # if len(comments) > 0:
    #     for i in range(len(comments)):
    #         comments[i] = re.sub(r"[^a-zA-Z\s]+", '', comments[i])
    pattern = r'<div class="title">(.+?)</div>'
    labels = re.findall(pattern, res.text)
    return comments, labels


In [ ]:
def game_data():
    infor_dict = {
        'game_name': [],
        'game_comment': [],
        'game_label': []
    }
    games = []
    games_type = [i for i in range(0, 2001, 100)]
    for name in games_type:
        games += get_games(name)
    print(len(games), 'total games')
    for i in range(len(games)):
        comments, labels = get_comments_games(games[i])
        if i % 100 == 0:
            print(i, 'games be processed')
        for j in range(len(comments)):
            infor_dict['game_name'].append(games[i])
            infor_dict['game_comment'].append(comments[j])
            infor_dict['game_label'].append(labels[j])

    df = pd.DataFrame(infor_dict)
    print(df.shape[0], 'total data')
    df.to_csv('games.csv', index=False)

In [ ]:
movie_data()
game_data()

## 3. Data Preprocessing

Read data & remove empty value row & change type

In [2]:
def read_data(name):
    # print porcess
    print("Reading",name,"...")
    
    # read csv
    text_data = pd.read_csv(name, header=0)
    text_data.columns = ["name", "comment", "label"]

    # print process
    print("Before dealing with empty value, the shape is:",text_data.shape)

    # deal with empty value
    text_data["comment"].replace('',np.nan,inplace=True) # replace the empty content to nan
    text_data = text_data.dropna(axis=0, how='any') # remove row with nan value

    # change type
    text_data = text_data.astype(str)

    # print process
    print("After dealing with empty value, the shape is:",text_data.shape)
    print("The head of data is: ")
    print(text_data.head(),end='\n\n')

    return text_data

movie = read_data("movies.csv")
game = read_data("games.csv")

Reading movies.csv ...
Before dealing with empty value, the shape is: (14050, 3)
After dealing with empty value, the shape is: (13735, 3)
The head of data is: 
      name                                            comment   label
0  juniper  Filmmakers should showcase Rampling’s indomita...  rotten
1  juniper  Savilles directorial debut is solid just like ...   fresh
2  juniper  It feels personal in a lot of the details and ...   fresh
3  juniper  We've seen this a million times -- but not wit...   fresh
4  juniper  Juniper takes some time to get where it is goi...   fresh

Reading games.csv ...
Before dealing with empty value, the shape is: (11721, 3)
After dealing with empty value, the shape is: (11115, 3)
The head of data is: 
  name                                            comment            label
0  730  After  years playing it I didnt improve my ski...      Recommended
1  730                                     I NOT CAN PLAY  Not Recommended
2  730  Your team in every random c

Process label

In [3]:
print("Processing labes")
game.replace(['Recommended', 'Not Recommended'], ['Positive','Negative'], regex=True , inplace=True)
movie.replace(['fresh', 'rotten'], ['Positive','Negative'], regex=True , inplace=True)
print("Processed labes")

Processing labes
Processed labes


Open abbreviation

In [4]:
contractions = {
    "aight": "alright",
    "ain't": "am not",
    "amn't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "daren't": "dare not",
    "daresn't": "dare not",
    "dasn't": "dare not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "d'ye": "do you",
    "e'er": "ever",
    "everybody's": "everybody is",
    "everyone's": "everyone is",
    "finna": "fixing to",
    "g'day": "good day",
    "gimme": "give me",
    "giv'n": "given",
    "gonna": "going to",
    "gon't": "go not",
    "gotta": "got to",
    "hadn't": "had not",
    "had've": "had have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'dn't've'd": "he would not have had",
    "he'll": "he will",
    "he's": "he is",
    "he've": "he have",
    "how'd": "how would",
    "howdy": "how do you do",
    "how'll": "how will",
    "how're": "how are",
    "I'll": "I will",
    "I'm": "I am",
    "I'm'a": "I am about to",
    "I'm'o": "I am going to",
    "innit": "is it not",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'll": "it will",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "may've": "may have",
    "methinks": "me thinks",
    "mightn't": "might not",
    "might've": "might have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "must've": "must have",
    "needn't": "need not",
    "ne'er": "never",
    "o'clock": "of the clock",
    "o'er": "over",
    "ol'": "old",
    "oughtn't": "ought not",
    "'s": "is",
    "shalln't": "shall not",
    "shan't": "shall not",
    "she'd": "she would",
    "she'll": "she shall",
    "she'll": "she will",
    "she's": "she has",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "somebody's": "somebody has",
    "somebody's": "somebody is",
    "someone's": "someone has",
    "someone's": "someone is",
    "something's": "something has",
    "something's": "something is",
    "so're": "so are",
    "that'll": "that shall",
    "that'll": "that will",
    "that're": "that are",
    "that's": "that has",
    "that's": "that is",
    "that'd": "that would",
    "that'd": "that had",
    "there'd": "there had",
    "there'd": "there would",
    "there'll": "there shall",
    "there'll": "there will",
    "there're": "there are",
    "there's": "there has",
    "there's": "there is",
    "these're": "these are",
    "these've": "these have",
    "they'd": "they had",
    "they'd": "they would",
    "they'll": "they shall",
    "they'll": "they will",
    "they're": "they are",
    "they're": "they were",
    "they've": "they have",
    "this's": "this has",
    "this's": "this is",
    "those're": "those are",
    "those've": "those have",
    "'tis": "it is",
    "to've": "to have",
    "'twas": "it was",
    "wanna": "want to",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd": "we would",
    "we'd": "we did",
    "we'll": "we shall",
    "we'll": "we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'd": "what did",
    "what'll": "what shall",
    "what'll": "what will",
    "what're": "what are",
    "what're": "what were",
    "what's": "what has",
    "what's": "what is",
    "what's": "what does",
    "what've": "what have",
    "when's": "when has",
    "when's": "when is",
    "where'd": "where did",
    "where'll": "where shall",
    "where'll": "where will",
    "where're": "where are",
    "where's": "where has",
    "where's": "where is",
    "where's": "where does",
    "where've": "where have",
    "which'd": "which had",
    "which'd": "which would",
    "which'll": "which shall",
    "which'll": "which will",
    "which're": "which are",
    "which's": "which has",
    "which's": "which is",
    "which've": "which have",
    "who'd": "who would",
    "who'd": "who had",
    "who'd": "who did",
    "who'd've": "who would have",
    "who'll": "who shall",
    "who'll": "who will",
    "who're": "who are",
    "who's": "who has",
    "who's": "who is",
    "who's": "who does",
    "who've": "who have",
    "why'd": "why did",
    "why're": "why are",
    "why's": "why has",
    "why's": "why is",
    "why's": "why does",
    "won't": "will not",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd've": "you all would have",
    "y'all'dn't've'd": "you all would not have had",
    "y'all're": "you all are",
    "you'd": "you had",
    "you'd": "you would",
    "you'll": "you shall",
    "you'll": "you will",
    "you're": "you are",
    "you're": "you are",
    "you've": "you have",
    " u ": "you",
    " ur ": "your",
    " n ": "and"
}

def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x

def apply_cont_to_exp(df):
    df['comment'] = df['comment'].apply(lambda x: cont_to_exp(x))
    return df

movie = apply_cont_to_exp(movie)
game = apply_cont_to_exp(game)

Convert spaces to single space, Remove quotation marks, Remove the newline character, Remove br

In [5]:
def remove_special_label(df):
    df.replace(['\ +', '"','\n','\r','/br[a-z]+br/g','/b[a-z]+b/g','RT'], [' ',' ','','','','',''], regex=True , inplace=True)
    return df

game = remove_special_label(game)

movie = remove_special_label(movie)

Cleaning operations

In [6]:
def preprocess_text(df):
    print("Converting to lower letter")
    df['comment'] = df['comment'].apply(lambda x: x.lower())

    # Remove HTML
    print("Removing HTML")
    df['comment'] = df['comment'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
    
    # Remove Emails
    print("Removing emails")
    df['comment'] = df['comment'].apply(lambda x: re.sub(
        r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x))

    # # Remove urls
    print("Removing urls")
    df['comment'] = df['comment'].apply(lambda x: re.sub(
        r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))
    
    # Create a spacy Doc object
    print("Removing stop words")
    df['comment'] = df['comment'].apply(lambda x: ' '.join(
        [t for t in x.split() if t not in STOP_WORDS]))
    
    # Removal of special chars and punctuation
    print("Removing special chars")
    df['comment'] = df['comment'].apply(
        lambda x: re.sub('[^\w]', ' ', x))
    
    # Removing multiple spaces
    print("Removing multi spaces")
    df['comment'] = df['comment'].apply(lambda x: ' '.join(x.split()))

    # count words frequency
    print("Counting words")
    frequency = pd.Series(' '.join(df['comment']).split()).value_counts()

    # Remove common words
    print("Removing common words")
    common = frequency[:10]
    df['comment'] = df['comment'].apply(lambda x: " ".join([t for t in x.split() if t not in common]))

    # Remove rare words
    print("Removing rare words")
    rare = frequency[-10:]
    df['comment'] = df['comment'].apply(lambda x: " ".join([t for t in x.split() if t not in rare]))
    
    return df

print("Processing game data\n")
game = preprocess_text(game)
print("\nProcessing movie data\n")
movie = preprocess_text(movie)

Processing game data

Converting to lower letter
Removing HTML
Removing emails
Removing urls
Removing stop words
Removing special chars
Removing multi spaces
Counting words
Removing common words
Removing rare words

Processing movie data

Converting to lower letter
Removing HTML


C:\Users\wangk\AppData\Local\Temp\ipykernel_38056\1505595802.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['comment'] = df['comment'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())


Removing emails
Removing urls
Removing stop words
Removing special chars
Removing multi spaces
Counting words
Removing common words
Removing rare words


Vectorization comments

In [7]:
nlp = spacy.load('en_core_web_sm')

def get_vec(x):
    doc = nlp(x)
    return doc.vector.reshape(1, -1)

def vec_data(df):
    df['comment'] = df['comment'].apply(lambda x: get_vec(x))
    return df

game = vec_data(game)
movie = vec_data(movie)

## 4. Explore data before training

## 5. Train SVM